In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
triplets_file ='https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file ='https://static.turi.com/datasets/millionsong/song_data.csv'

In [8]:
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

In [14]:
song_df_2 = pd.read_csv(songs_metadata_file)
song_df = pd.merge(song_df_1,song_df_2.drop_duplicates(['song_id']), on="song_id",how="left")

In [41]:
song_grouped =song_df.groupby(['song_id']).agg({'listen_count':'count'}).reset_index()
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage']=song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values(['listen_count', 'song_id'],ascending = [0,1])


,song_id,listen_count,percentage
2220,SOFRQTD12A81C233C0,8277,0.41385
317,SOAUWYT12A81C206F1,7032,0.35160
352,SOAXGDH12A8C13F8A1,6949,0.34745
614,SOBONKR12A58A7A7E0,6412,0.32060
7416,SOSXLTC12AF72A7F54,6145,0.30725
5531,SONYKOW12AB01849C9,5841,0.29205
1664,SOEGIYH12A6D4FC0E3,5385,0.26925
4448,SOLFXKT12AB017E3E0,4795,0.23975
1334,SODJWHY12A8C142CCE,4758,0.23790
2115,SOFLJQZ12A6D4FADA6,4548,0.22740


In [38]:
users = song_df['user_id'].unique()
len(users) ## return 365 unique users
songs = song_df['song_id'].unique()
len(songs) ## return 5151 unique songs

10000

In [34]:
train_data, test_data = train_test_split(song_df, test_size= 0.20, random_state=0)

In [30]:
#Class for Popularity based Recommender System modelclass
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
#Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
#Get a count of user_ids for each unique song as recommendation score
        train_data_grouped=train_data.groupby([self.item_id]).agg({self.user_id:'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id':'score'},inplace=True)
#Sort the songs based upon recommendation score
        train_data_sort =train_data_grouped.sort_values(['score', self.item_id],ascending = [0,1])
#Generate a recommendation rank based upon score
        train_data_sort['Rank'] =train_data_sort['score'].rank(ascending=0, method='first')
#Get the top 10 recommendations
        self.popularity_recommendations =train_data_sort.head(10)
#Use the popularity based recommender system model to make recommendations
    def recommend(self, user_id):
        user_recommendations =self.popularity_recommendations
#Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
#Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        return user_recommendations


In [42]:
pm = popularity_recommender_py()
pm.create(train_data, 'user_id', 'song_id')
#user the popularity model to make some prediction
user_id = users[5]
pm.recommend(user_id)

,user_id,song_id,score,Rank
2220,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOFRQTD12A81C233C0,6630,1.0
317,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOAUWYT12A81C206F1,5639,2.0
352,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOAXGDH12A8C13F8A1,5592,3.0
614,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOBONKR12A58A7A7E0,5143,4.0
7416,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOSXLTC12AF72A7F54,4938,5.0
5531,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SONYKOW12AB01849C9,4627,6.0
1664,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOEGIYH12A6D4FC0E3,4368,7.0
4448,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOLFXKT12AB017E3E0,3835,8.0
1334,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SODJWHY12A8C142CCE,3819,9.0
2115,4bd88bfb25263a75bbdd467e74018f4ae570e5df,SOFLJQZ12A6D4FADA6,3707,10.0


In [44]:
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        
        return user_items
        
    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
            
        return item_users
        
    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
            
        return all_items
        
    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):
            
        ####################################
        #Get users for all songs in user_songs.
        ####################################
        user_songs_users = []        
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))
            
        ###############################################
        #Initialize the item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)
           
        #############################################################
        #Calculate similarity between user songs and all unique songs
        #in the training data
        #############################################################
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())
            
            for j in range(0,len(user_songs)):       
                    
                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]
                    
                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)
                
                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)
                    
                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0
                    
        
        return cooccurence_matrix

    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pandas.DataFrame(columns=columns)
         
        #Fill the dataframe with top 10 item based recommendations
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        
        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
 
    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):
        
        ########################################
        #A. Get all unique songs for this user
        ########################################
        user_songs = self.get_user_items(user)    
            
        print("No. of unique songs for the user: %d" % len(user_songs))
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return df_recommendations
    
    #Get similar items to given items
    def get_similar_items(self, item_list):
        
        user_songs = item_list
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
         
        return df_recommendations

In [48]:
is_model = item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song_id')

In [ ]:
user_id = users[5]
user_items = is_model.get_user_items(user_id)
#
print("---------------------------------------------------------")
print("Training data songs for the user userid: %s:" %user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)
print("---------------------------------------------------------------------")
print("Recommendation process going on:")
print("---------------------------------------------------------------------")
#Recommend songs for the user using personalized model
is_model.recommend(user_id)

---------------------------------------------------------
Training data songs for the user userid: 4bd88bfb25263a75bbdd467e74018f4ae570e5df:
------------------------------------------------------------------------------------
SOKOXWU12AF72AD1BC
SOMNTMT12A8C1400F6
SOUSQCN12A8C133302
SOXGQEM12AB0181D35
SOEPZQS12A8C1436C7
SOGEKGR12A6D4F81E8
SOWGXOP12A6701E93A
SODGVGW12AC9075A8D
SOVEUVC12A6310EAF1
---------------------------------------------------------------------
Recommendation process going on:
---------------------------------------------------------------------
No. of unique songs for the user: 9
no. of unique songs in the training set: 10000
